# Data Engineering - ETL


## Objectives

The aims of this task are:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


As working environment I am using Watson Studio in
Cloud Pak for Data

Python and several Python libraries will be used. 
Some of these libraries are already installed in my lab environment and others may need to be installed. The cells below will install these libraries when executed.


In [1]:
#!pip install glob
#!pip install pandas
#!pip install requests
#!pip install datetime

## Imports

Importing any additional libraries I may need.


In [2]:
import glob
import pandas as pd
from datetime import datetime

For this task I will be using given dataset and fluctuations in the exchange rate will be prevented by working with static data. As a next task after testing I will be connecting to webpage in order to get udated rates.

In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2022-01-21 10:09:12--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2022-01-21 10:09:13 (115 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2022-01-21 10:09:14--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tran

## Extract


### JSON Extract Function

This function will extract JSON files.


In [4]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Defining the extract function that finds JSON file `bank_market_cap_1.json` and calling the function created above to extract data from them. Storing the data in a `pandas` dataframe. Using the following list for the columns.


columns=['Name','Market Cap (US$ Billion)']

In [5]:
def extract():
    # Write your code here
    # creating an empty data frame to hold extracted data
    extracted_data = pd.DataFrame(columns=['Name','Market Cap (US$ Billion)'])
    
    #process all json files
    for jsonfile in glob.glob('*.json'):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    return extracted_data

In [6]:
extracted_data_test = extract()
extracted_data_test

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
100,Ping An Bank,37.993
101,Standard Chartered,37.319
102,United Overseas Bank,35.128
103,QNB Group,33.560


Loading the file <code>exchange_rates.csv</code> as a dataframe and finding the exchange rate for British pounds with the symbol <code>GBP</code>, storing it in the variable  <code>exchange_rate</code>. To set the parameter  <code>index_col</code> to 0.


In [7]:
exchange_rates_data = pd.read_csv('exchange_rates.csv', index_col = 0)
# exchange_rates_data

In [8]:
exchange_rates_data.loc[['GBP']]

,Rates
GBP,0.732398


In [9]:
exchange_rate = float(exchange_rates_data.loc['GBP'])
exchange_rate

0.7323984208000001

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file finding the exchange rate of USD to GBP. Writeing a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [10]:
def transform(data):
    data['Market Cap (US$ Billion)'] = round(data['Market Cap (US$ Billion)']*exchange_rate,3)
    data.rename(columns={'Market Cap (US$ Billion)':'Market Cap (GBP£ Billion)'}, inplace = True)
    
    return data    
    

In [11]:
transformed_data_test = transform(extracted_data_test)
transformed_data_test

,Name,Market Cap (GBP£ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519
...,...,...
100,Ping An Bank,27.826
101,Standard Chartered,27.332
102,United Overseas Bank,25.728
103,QNB Group,24.579


## Load

Creating a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. 
To set `index` to `False`.


In [12]:
## Seting Paths
targetfile = 'bank_market_cap_gbp.csv'
tempfile_rowdata = 'bank_market_cap_gbp_rowdata.csv'
logfile = 'bank_market_cap_gbp_log.txt'

In [13]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile, index = False)    

In [14]:
# load(targetfile,transformed_data_test)

In [15]:
# df_test = pd.read_csv(targetfile)
# df_test

## Logging Function


Writing the logging function <code>log</code> to log my data:


In [16]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'      # Hour-Minute-Second-Monthname-Day-Year
    now = datetime.now()                        # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(logfile, 'a') as f:               # logfile is defined earlier in "Set path"
        f.write(timestamp + ',' + message + '\n')   

## Running the ETL Process


Loging the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [17]:
log('ETL Job Started')
log('Extract phase Started')

### Extract


Using the function <code>extract</code>, and printing the first 5 rows:


In [18]:
extracted_data = extract()
extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Loging the data as <code>"Extract phase Ended"</code>


In [19]:
log('Extract phase Ended')

### Transform


Loging the following  <code>"Transform phase Started"</code>


In [20]:
log('Transform phase Started')

Using the function <code>transform</code> and printing the first 5 rows of the output:


In [21]:
transformed_data = transform(extracted_data)
transformed_data.head()

,Name,Market Cap (GBP£ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


Loging my data <code>"Transform phase Ended"</code>


In [22]:
log('Transform phase Ended')

### Load


Loging the following `"Load phase Started"`.


In [23]:
log('Load phase Started')

Calling the load function


In [24]:
load(targetfile, transformed_data)

In [25]:
load_check = pd.read_csv(targetfile)
load_check

,Name,Market Cap (GBP£ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519
...,...,...
100,Ping An Bank,27.826
101,Standard Chartered,27.332
102,United Overseas Bank,25.728
103,QNB Group,24.579


Loging the following `"Load phase Ended"`.


In [26]:
load_check_log = pd.read_csv(logfile)
load_check_log.tail(8)

,10:09:16-Jan-21-2022,ETL Job Started
0,10:09:16-Jan-21-2022,Extract phase Started
1,10:09:16-Jan-21-2022,Extract phase Ended
2,10:09:16-Jan-21-2022,Transform phase Started
3,10:09:16-Jan-21-2022,Transform phase Ended
4,10:09:16-Jan-21-2022,Load phase Started
